In [1]:
# huggingface transformer vit
import torch
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import requests
from io import BytesIO
import os, json

In [2]:
def use_huggingface_vit():
    '''huggingface transformers vit'''
    try:
        from transformers import ViTForImageClassification, ViTImageProcessor
        # 모델과 프로세스 로드
        model_name = 'google/vit-base-patch16-224'
        processor = ViTImageProcessor.from_pretrained(model_name)
        model = ViTForImageClassification.from_pretrained(model_name)
        model.eval()
        print(f'\n[모델 정보]')
        print(f'파라메터수 : { sum(p.numel()  for p in model.parameters())}')
        print(f'클래스 수 : { model.config.num_channels}')
        print(f'이미지 크기 : { model.config.image_size}')
        print(f'패치 크기 : { model.config.patch_size}')
        print(f'히든 크기 : { model.config.hidden_size}')
        print(f'레이어 수 : { model.config.num_hidden_layers}')
        print(f'어텐션 해드 수 : { model.config.num_attention_heads}')
        return model, processor
    except Exception as e:
        print(f' hugging face vit 로드 실패 : {e}')
        return None, None

In [ ]:
# timm 라이브러리를 사용한 vit
def use_timm_vit():
    '''timm 라이브러리 vit'''    
    import timm
    # 사용가능한 vit 모델 목록
    vit_models = timm.list_models('vit*', pretrained=True)
    for model_name in vit_models:
        if 'vit_base_patch16_224' in model_name:
            print(f'    - {model_name}')
    print(f'총   {len(vit_models)}개 모델')

    model = timm.create_model('vit_base_patch16_224',pretrained=True)
    print(f"실제 다운로드 모델명 : vit_base_patch16_224.{model.default_cfg['tag']}")

    # timm의 데이터 설정 가져오기
    data_config = timm.data.resolve_model_data_config(model)
    transform = timm.data.create_transform(**data_config,is_training = False)
    return  model, transform

def classify_image_hf(model, processor, image):
    """Hugging Face 모델로 이미지 분류"""
    
    if model is None:
        print("  모델이 로드되지 않았습니다.")
        return None
    
    # 이미지 전처리
    inputs = processor(images=image, return_tensors="pt")
    print(f"\n[전처리된 입력]")
    print(f"  pixel_values shape: {inputs['pixel_values'].shape}")
    
    # 추론
    with torch.no_grad():
        outputs = model(**inputs)
    
    logits = outputs.logits
    print(f"\n[모델 출력]")
    print(f"  logits shape: {logits.shape}")
    
    # Top-5 예측
    probs = F.softmax(logits, dim=-1)
    top5_probs, top5_indices = torch.topk(probs, 5)
    
    print(f"\n[Top-5 예측 결과]")
    for i, (prob, idx) in enumerate(zip(top5_probs[0], top5_indices[0])):
        label = model.config.id2label[idx.item()]
        print(f"  {i+1}. {label}: {prob.item():.4f} ({prob.item()*100:.2f}%)")
    
    return top5_probs[0], top5_indices[0]

def classify_image_timm(model, transform, image):
    """timm 모델로 이미지 분류""" 
    
    if model is None:
        print("  모델이 로드되지 않았습니다.")
        return None
    
    # 이미지 전처리
    img_tensor = transform(image).unsqueeze(0)
    print(f"\n[전처리된 입력]")
    print(f"  tensor shape: {img_tensor.shape}")
    
    # 추론
    with torch.no_grad():
        outputs = model(img_tensor)
    
    print(f"\n[모델 출력]")
    print(f"  outputs shape: {outputs.shape}")
    
    # Top-5 예측
    probs = F.softmax(outputs, dim=-1)
    top5_probs, top5_indices = torch.topk(probs, 5)
    
    # ImageNet 클래스 이름 로드
    try:
        url = "https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt"
        response = requests.get(url, timeout=10)
        categories = [s.strip() for s in response.text.splitlines()]
        
        print(f"\n[Top-5 예측 결과]")
        for i, (prob, idx) in enumerate(zip(top5_probs[0], top5_indices[0])):
            label = categories[idx.item()] if idx.item() < len(categories) else f"class_{idx.item()}"
            print(f"  {i+1}. {label}: {prob.item():.4f} ({prob.item()*100:.2f}%)")
            
    except Exception as e:
        print(f"\n[Top-5 예측 결과 (인덱스)]")
        for i, (prob, idx) in enumerate(zip(top5_probs[0], top5_indices[0])):
            print(f"  {i+1}. class_{idx.item()}: {prob.item():.4f} ({prob.item()*100:.2f}%)")
    
    return top5_probs[0], top5_indices[0]


if __name__=='__main__':
    # 모델, 프리프로세스 로드(huggingfase, timm)
    hf_model, hf_process = use_huggingface_vit()
    timm_model, timm_process = use_timm_vit()
    # 샘플이미지 확보
    # hf에서 받은 모델, timm에서 받은 모델로 전용 추론함수에 넣어서 결과를 확인(ex classify_image_timm((timm_model,timm_process,img))
    


[모델 정보]
파라메터수 : 86567656
클래스 수 : 3
이미지 크기 : 224
패치 크기 : 16
히든 크기 : 768
레이어 수 : 12
어텐션 해드 수 : 12
    - vit_base_patch16_224.augreg2_in21k_ft_in1k
    - vit_base_patch16_224.augreg_in1k
    - vit_base_patch16_224.augreg_in21k
    - vit_base_patch16_224.augreg_in21k_ft_in1k
    - vit_base_patch16_224.dino
    - vit_base_patch16_224.mae
    - vit_base_patch16_224.orig_in21k
    - vit_base_patch16_224.orig_in21k_ft_in1k
    - vit_base_patch16_224.sam_in1k
    - vit_base_patch16_224_miil.in21k
    - vit_base_patch16_224_miil.in21k_ft_in1k
총   333개 모델
실제 다운로드 모델명 : vit_base_patch16_224.augreg2_in21k_ft_in1k


In [18]:
from PIL import Image
import requests
from io import BytesIO

image_path = "C:/python_src/7.Multimodal/251215/download_img/Cat.jpg"
image = Image.open(image_path).convert("RGB")

print("\n===== timm ViT 결과 =====")
classify_image_timm(timm_model, timm_process, image)

print("\n===== HuggingFace ViT 결과 =====")
classify_image_hf(hf_model, hf_process, image)


===== timm ViT 결과 =====

[전처리된 입력]
  tensor shape: torch.Size([1, 3, 224, 224])

[모델 출력]
  outputs shape: torch.Size([1, 1000])

[Top-5 예측 결과]
  1. tiger cat: 0.6027 (60.27%)
  2. Egyptian cat: 0.1188 (11.88%)
  3. tabby: 0.0718 (7.18%)
  4. lynx: 0.0218 (2.18%)
  5. swab: 0.0161 (1.61%)

===== HuggingFace ViT 결과 =====

[전처리된 입력]
  pixel_values shape: torch.Size([1, 3, 224, 224])

[모델 출력]
  logits shape: torch.Size([1, 1000])

[Top-5 예측 결과]
  1. tiger cat: 0.4328 (43.28%)
  2. tabby, tabby cat: 0.2598 (25.98%)
  3. Egyptian cat: 0.1613 (16.13%)
  4. broom: 0.0122 (1.22%)
  5. swab, swob, mop: 0.0077 (0.77%)


(tensor([0.4328, 0.2598, 0.1613, 0.0122, 0.0077]),
 tensor([282, 281, 285, 462, 840]))

In [20]:
if __name__=='__main__':
    from glob import glob
    # 모델, 프리프로세스 로드(huggingface, timm)
    hf_model, hf_process = use_huggingface_vit()
    timm_model, timm_process = use_timm_vit()
    # 셈플이미지 확보
    # hf 에서 받은 모델, timm에서 받은 모델로 전용 추론함수에 넣어서 결과를 확인 (ex classify_image_timm(timm_model,timm_process,img)   )
    file_paths = 'C:/python_src/7.Multimodal/251215/download_img'
    files = glob(file_paths+'/*.jpg')
    print('\n\n[ timm 추론]...')
    for file in files:
        test_img = Image.open(file).convert('RGB')
        classify_image_timm(timm_model,timm_process,test_img)     


[모델 정보]
파라메터수 : 86567656
클래스 수 : 3
이미지 크기 : 224
패치 크기 : 16
히든 크기 : 768
레이어 수 : 12
어텐션 해드 수 : 12
    - vit_base_patch16_224.augreg2_in21k_ft_in1k
    - vit_base_patch16_224.augreg_in1k
    - vit_base_patch16_224.augreg_in21k
    - vit_base_patch16_224.augreg_in21k_ft_in1k
    - vit_base_patch16_224.dino
    - vit_base_patch16_224.mae
    - vit_base_patch16_224.orig_in21k
    - vit_base_patch16_224.orig_in21k_ft_in1k
    - vit_base_patch16_224.sam_in1k
    - vit_base_patch16_224_miil.in21k
    - vit_base_patch16_224_miil.in21k_ft_in1k
총   333개 모델
실제 다운로드 모델명 : vit_base_patch16_224.augreg2_in21k_ft_in1k


[ timm 추론]...

[전처리된 입력]
  tensor shape: torch.Size([1, 3, 224, 224])

[모델 출력]
  outputs shape: torch.Size([1, 1000])

[Top-5 예측 결과]
  1. bulbul: 0.3902 (39.02%)
  2. water ouzel: 0.0643 (6.43%)
  3. brambling: 0.0623 (6.23%)
  4. goldfinch: 0.0567 (5.67%)
  5. jacamar: 0.0466 (4.66%)

[전처리된 입력]
  tensor shape: torch.Size([1, 3, 224, 224])

[모델 출력]
  outputs shape: torch.Size([1, 1000])